In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression , LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, classification_report, mean_absolute_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import joblib as jb
import gradio as gr
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_csv(r"C:\Users\DELL\Study\Semester 5\Machine Learning\Project\Sleep_health_and_lifestyle_dataset.csv")

In [3]:
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [5]:
df.isnull().sum()

Person ID                    0
Gender                       0
Age                          0
Occupation                   0
Sleep Duration               0
Quality of Sleep             0
Physical Activity Level      0
Stress Level                 0
BMI Category                 0
Blood Pressure               0
Heart Rate                   0
Daily Steps                  0
Sleep Disorder             219
dtype: int64

In [6]:
df.columns

Index(['Person ID', 'Gender', 'Age', 'Occupation', 'Sleep Duration',
       'Quality of Sleep', 'Physical Activity Level', 'Stress Level',
       'BMI Category', 'Blood Pressure', 'Heart Rate', 'Daily Steps',
       'Sleep Disorder'],
      dtype='object')

In [7]:
df_new = df.drop(['Person ID', 'Gender', 'Age', 'Occupation','Quality of Sleep','BMI Category', 'Blood Pressure', 'Sleep Disorder'], axis=1)

In [8]:
df_new.head()

,Sleep Duration,Physical Activity Level,Stress Level,Heart Rate,Daily Steps
0,6.1,42,6,77,4200
1,6.2,60,8,75,10000
2,6.2,60,8,75,10000
3,5.9,30,8,85,3000
4,5.9,30,8,85,3000


In [26]:
min_row = df_new.loc[df['Stress Level'].idxmin()]

print(min_row)

Sleep Duration                7.5
Physical Activity Level      45.0
Stress Level                  3.0
Heart Rate                   70.0
Daily Steps                8000.0
Name: 50, dtype: float64


In [9]:
df_new.columns

Index(['Sleep Duration', 'Physical Activity Level', 'Stress Level',
       'Heart Rate', 'Daily Steps'],
      dtype='object')

In [10]:
x = df_new[['Sleep Duration', 'Physical Activity Level', 'Heart Rate', 'Daily Steps']]
y = df_new['Stress Level']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [11]:
scaled = StandardScaler()
x_train = scaled.fit_transform(x_train)
x_test = scaled.transform(x_test)

In [12]:
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)


In [22]:
prediction1 = model.predict([[8, 60, 86, 10000]])
print("Predicted Stress Level:", prediction1)

Predicted Stress Level: [8]


In [13]:
r2 = r2_score(y_test, y_pred)
r2

0.9061113007852509

In [16]:
jb.dump(model, r'C:\Users\DELL\Study\Semester 5\Machine Learning\Project\stress_model.pkl')

['C:\\Users\\DELL\\Study\\Semester 5\\Machine Learning\\Project\\stress_model.pkl']

In [21]:
# 1. Load the trained model
model = jb.load(r'C:\Users\DELL\Study\Semester 5\Machine Learning\Project\stress_model.pkl')

# 2. Define the Prediction Function
# This function takes inputs from the website, runs the model, and returns the result
def predict_stress(sleep_hours, activity_minutes, heart_rate, daily_steps):
    # Prepare the input array (must match the order of training features)
    features = np.array([[ sleep_hours, activity_minutes, heart_rate, daily_steps]])
    
    # Make prediction
    prediction = model.predict(features)
    
    # Return the result (rounded to 1 decimal place)
    return round(prediction[0], 1)

# 3. Create the Gradio Interface
# We map the function inputs to Sliders for a nice UI
interface = gr.Interface(
    fn=predict_stress,
    inputs=[
        gr.Number(minimum=0, maximum=12, label="Sleep Duration (Hours)"),
        gr.Number(minimum=0, maximum=120, label="Physical Activity (Minutes/Day)"),
        gr.Number(minimum=40, maximum=120, label="Heart Rate (bpm)"),
        gr.Number(minimum=0, maximum=30000, label="Daily Steps")
    ],
    outputs=gr.Number(label="Predicted Stress Level (1-10)"),
    title="Stress Level Predictor",
    description="Enter your daily health metrics to predict your stress level based on the Sleep Health Dataset."
)

# 4. Launch the App
if __name__ == "__main__":
    interface.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\blocks.py", line 2121, in process_api
    inputs = await self.preprocess_data(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\blocks.py", line 1768, in preprocess_data
    processed_value = block.preprocess(inputs_cached)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\DELL\AppData\Local\Programs\Python\Python